In [1]:
import psycopg2
import sys

In [2]:
con = psycopg2.connect(database='twitter', user='learner')

In [3]:
cur = con.cursor()

In [4]:
cur.execute('SELECT version()')
ver = cur.fetchone()
print ver

('PostgreSQL 9.3.10 on x86_64-unknown-linux-gnu, compiled by gcc (Ubuntu 4.8.2-19ubuntu1) 4.8.2, 64-bit',)


In [5]:
cur.execute("DELETE FROM tweet_usermention")
cur.execute("DELETE FROM tweet_entities")
cur.execute("DELETE FROM tweets")
cur.execute("DELETE FROM users")
con.commit()


In [6]:
users = {}
tweets = {}
retweets = {}
tweet_usermentions = []
tweet_entities = []

In [7]:
def append_field(list_name, json, field_name):
    if field_name in json:
        list_name.append(json[field_name])
    else: 
        list_name.append(None)

In [8]:
def parse_user(json):
    user = []
    
    user.append(json['id'])
    user.append(json['screen_name'])
    user.append(json['name'])
    append_field(user, json, 'created_at')
    append_field(user, json, 'description')
    append_field(user, json, 'retweet_count')
    append_field(user, json, 'favorite_count')
    append_field(user, json, 'friends_count')
    append_field(user, json, 'followers_count')
    append_field(user, json, 'statuses_count')
    append_field(user, json, 'geo_enabled')
    append_field(user, json, 'time_zone')
    append_field(user, json, 'profile_image_url')
    
    if user[0] in users:
        if users[user[0]][12] is None:
            users[user[0]] = user
    else:
        users[user[0]] = user

In [9]:
def parse_tweet(list_name, json):
    tweet = []
   
    tweet.append(json['id'])
    tweet.append(json['created_at'])
    
    if 'retweeted_status' in json:
        tweet.append(json['retweeted_status']['id'])
    else:
        tweet.append(None)
        
    append_field(tweet, json, 'source')
    append_field(tweet, json, 'text')
    append_field(tweet, json, 'retweet_count')
    append_field(tweet, json, 'favorite_count')
    tweet.append(json['user']['id'])
        
    append_field(tweet, json, 'lang')
    if 'geo' in json:
        tweet.append(json['geo']['type'])
        tweet.append(json['geo']['coordinates'][0])
        tweet.append(json['geo']['coordinates'][1])
    else:
        tweet.append(None)
        tweet.append(None)
        tweet.append(None)
        
    if 'in_reply_to_user' in json:
        tweet.append(json['in_reply_to_user']['id'])
    else:
        tweet.append(None)
    
    list_name[tweet[0]] = tweet
    
    for entity in json['entities']:
        tweet_entities.append((tweet[0], entity))     
    
    for user in json['user_mentions']:
        tweet_usermentions.append((tweet[0], user["id"]))

In [10]:
def parse(json):
    parse_user(json['user'])
    if 'retweeted_status' in json:
        parse_user(json['retweeted_status']['user'])
        
    for user in json['user_mentions']:
        parse_user(user)
        
    if 'in_reply_to_user' in json:
        parse_user(json['in_reply_to_user'])
        
    if 'retweeted_status' in json:
        parse_tweet(retweets, json['retweeted_status'])
        
    parse_tweet(tweets, json)

In [11]:
cur = con.cursor()

In [12]:
def load_file(tweets_data_path):
    tweets_file = open(tweets_data_path, "r")
    for line in tweets_file:
        try:
            tweet = json.loads(line)
            parse(tweet)

        except:
            print "Unexpected error:", sys.exc_info()[0]
            raise

In [13]:
import json
from pprintpp import pprint as pp

load_file('../data/tweets.json')
load_file('../data/timeline.json')

for tweet_id in retweets:
    tweets.pop(tweet_id, None)

print len(users)
print len(tweets)
print len(retweets)
print len(set(tweet_entities))
print len(set(tweet_usermentions))

3403
5367
3701
23578
7416


In [14]:
query = """INSERT INTO users (user_id, 
            screen_name, 
            name,
            created_at,
            description,
            retweet_count,
            favorite_count,
            friends_count,
            followers_count,
            statuses_count,
            geo_enabled,
            time_zone,
            profile_image_url)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""

cur.executemany(query, users.values())

In [15]:
query = """
       INSERT INTO tweets (tweet_id, 
            created_at, 
            tweet_id_retweeted,
            source,
            text,
            retweet_count,
            favorite_count,
            user_id,
            lang,
            geo_type,
            geo_x,
            geo_y,
            reply_to_user_id)
        VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
        """

cur.executemany(query, retweets.values())
cur.executemany(query, tweets.values())

In [16]:
query = """INSERT INTO tweet_usermention (tweet_id, 
            user_id)
        VALUES (%s,%s)"""

cur.executemany(query, set(tweet_usermentions))

In [17]:
query = """INSERT INTO tweet_entities (tweet_id, 
            entity)
        VALUES (%s,%s)"""

cur.executemany(query, set(tweet_entities))

In [18]:
con.commit()
con.close()